In [87]:
import pandas as pd
import re
import spacy
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# Charger le modèle linguistique français de spaCy
nlp = spacy.load("fr_core_news_sm")

In [88]:
# Charger les données
DataFrame = pd.read_excel("/Users/charlescook/Desktop/Avis_bar.xlsx")

In [89]:
# Supprimer les lignes où il manque des avis ou des notes 
DataFrame = DataFrame.dropna(subset=["Avis", "Note"])
DataFrame["Note"] = pd.to_numeric(DataFrame["Note"], errors="coerce")


In [90]:
# Nettoyage des avis
def nettoyage_prealable(text):
    # Conversion en minuscule
    text = text.lower()
    # Supprimer les caractères spéciaux
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    # Retirer les espaces multiples
    text = re.sub(r"\s+", " ", text).strip()
    return text

DataFrame["Avis_Nettoyés"] = DataFrame["Avis"].apply(nettoyage_prealable)

In [91]:
# Traitement avec spaCy : tokenisation, suppression des stopwords et lemmatisation
def spacy_process(text):
    doc = nlp(text) # Création du doc qui contient des tokens, des informations grammaticales sur le mot
    tokens = [
        token.lemma_  # Lemmatisation (transformationd des verbes en infinitif par exemple)
        for token in doc
        if not token.is_stop and not token.is_punct  # Retirer stopwords et ponctuation
    ]
    return tokens

DataFrame["Tokens"] = DataFrame["Avis_Nettoyés"].apply(spacy_process)

In [92]:
# Reconstituer les avis nettoyés et lemmatisés
DataFrame["Cleaned_Avis"] = DataFrame["Tokens"].apply(lambda x: " ".join(x))

In [93]:
DataFrame.to_excel("/Users/charlescook/Desktop/resultats_modifies_bar.xlsx", index=False)

In [94]:
# Configurer TextBlob pour le français
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

# Charger le fichier Excel modifié
file_path = "/Users/charlescook/Desktop/resultats_modifies_bar.xlsx"  # Remplacez par le chemin correct
colonne = pd.read_excel(file_path)

In [95]:
# Fonction pour analyser les sentiments avec TextBlob
def analyze_sentiment(text): # en entrée on prend un chaîne de caractères
    blob = tb(text) # objet qui analyse le texte notamment la polarité et la subjectivité
    return blob.sentiment # renvoie un tuple contenant polarité et subjectivité
# Appliquer l'analyse de sentiment
colonne["Sentiment"] = colonne["Cleaned_Avis"].apply(lambda x: analyze_sentiment(x) if isinstance(x, str) else None) 

In [96]:
# Séparer la polarité et la subjectivité dans deux colonnes distinctes
colonne["Polarité"] = colonne["Sentiment"].apply(lambda x: x[0] if x else None)  # Polarité : entre -1 et 1
colonne["Subjectivité"] = colonne["Sentiment"].apply(lambda x: x[1] if x else None)  # Subjectivité : entre 0 et 1

# Supprimer la colonne intermédiaire "Sentiment"
colonne = colonne.drop(columns=["Sentiment"])

# Sauvegarder les résultats dans un nouveau fichier Excel
output_path = "/Users/charlescook/Desktop/résultats_avec_sentiments.xlsx"  # Chemin pour le fichier de sortie
colonne.to_excel(output_path, index=False)

In [106]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_excel("/Users/charlescook/Desktop/résultats_avec_sentiments.xlsx")
documents = df["Avis"].dropna().tolist()

In [107]:
# Nettoyage et prétraitement

# Charger le modèle SpaCy français
nlp = spacy.load("fr_core_news_sm")

def nettoyer_avis(texte):
    doc = nlp(str(texte).lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

# Nettoyage des avis
textes_nettoyes = [nettoyer_avis(avis) for avis in documents]

In [108]:
# Vectorisation TF-IDF

vectorizer = TfidfVectorizer(max_features=200)
X = vectorizer.fit_transform(textes_nettoyes)

In [109]:
# Clustering avec KMeans

df = df.dropna(subset=["Avis"]).reset_index(drop=True)

# Définir le nombre de clusters
nombre_clusters = 2
kmeans = KMeans(n_clusters=nombre_clusters, random_state=42)
kmeans.fit(X)

# Ajouter les étiquettes de cluster dans le DataFrame
df["Cluster"] = kmeans.labels_

In [110]:
# Application de LDA pour les mots-clés par thème
lda = LatentDirichletAllocation(n_components=nombre_clusters, random_state=42)
lda.fit(X)

LatentDirichletAllocation(n_components=2, random_state=42)

In [111]:
# Affichage des mots-clés associés à chaque thème
print("\nMots-clés des thèmes identifiés par LDA :")
themes = []
for i, topic in enumerate(lda.components_):
    mots_importants = [vectorizer.get_feature_names_out()[index] for index in topic.argsort()[-5:]]
    themes.append(", ".join(mots_importants))
    print(f"Thème {i + 1}: {', '.join(mots_importants)}")


Mots-clés des thèmes identifiés par LDA :
Thème 1: serveur, accueil, boisson, prix, qualite
Thème 2: bon, bar, service, cocktail, ambiance


In [112]:
# Ajouter les thèmes dans le DataFrame
df["Thème associé"] = df["Cluster"].map(lambda x: themes[x])

In [113]:
# Exporter les résultats

df.to_excel("resultats_clustering.xlsx", index=False)

In [120]:
# Calcul des moyennes par cluster

DataFrame = pd.read_excel("resultats_clustering.xlsx")

def moyenne_ponderee_polarite(group):
    # Appliquer la formule de moyenne pondérée
    poids = 1 - group["Subjectivité"]
    return ((group["Polarité"] * poids).sum() / poids.sum())*100

# Calcul de la moyenne pondérée par cluster
resultats_pondérés = DataFrame.groupby("Thème associé", group_keys=False).apply(lambda group: moyenne_ponderee_polarite(group[["Polarité", "Subjectivité"]]))

print("Moyenne pondérée de la polarité par cluster :")
print(resultats_pondérés)

Moyenne pondérée de la polarité par cluster :
Cluster
0    27.507671
1     8.538433
dtype: float64


/var/folders/vt/4zt7370n2zddn4pc_s4zyb480000gn/T/ipykernel_79123/520481331.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultats_pondérés = DataFrame.groupby("Cluster", group_keys=False).apply(lambda group: moyenne_ponderee_polarite(group[["Polarité", "Subjectivité"]]))
